# Implementing a CNN in PyTorch

## Introduction

**Learning Objectives**
- How to create a CNN using PyTorch  
- When to stop training a neural network
- The challenges of training a CNN

Implementing a **CNN** in PyTorch is exactly the same as creating a vanilla **neural network**. The only difference is the **layers** that we use. As with much in machine learning, the code for the creation of the model is simple, it is the choices you make as the user that will define your success.

## Setup

First lets setup the notebook with the necessary imports:

In [ ]:
import torch # the PyTorch module
import torch.nn as nn # the neural network module
import torch.nn.functional as F # the functional module
import torch.optim as optim # the optimiser module

import warnings
warnings.filterwarnings("ignore") # ignore warnings

# Import the MNIST dataset
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np

# fix the numpy and torch seeds for reproducibility:
SEED = 999
np.random.seed(SEED)
torch.manual_seed(SEED);

And import the MNIST dataset again:

In [ ]:
# Normalise the data to have mean 0 and std 1 using dataset statistics
transform = transforms.Compose([
                          transforms.ToTensor(),    # convert the image to a tensor
                          transforms.Normalize((0.1307,), (0.3081,)) # normalise the image
                        ])

# Define a training data loader

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST( 'data', 
                    train=True, 
                    download=True,
                    transform = transform
                    ),
    batch_size=64, shuffle=True)

# Define a test data loader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST( 'data', 
                    train=False, 
                    download=True,
                    transform = transform
                    ),
    batch_size=1000, shuffle=True)

## Defining the CNN

Similarly to the previous exercise, we will use the `nn` module in PyTorch to define the layers of the network. The `nn` module has a `Sequential` class that allows us to define a network as a sequence of layers. We can also use the `nn` module to define the loss function and the optimiser.

As the MNIST images are greyscale, the **input channel is 1**, as there is only one channel, the pixel intensity between black and white. The output channel is 4, which means we are using 4 kernels/filters to extract features from the image. Each kernel has differently randomised weights, which will then learn different features of the image.

We also have a kernel size of 3, or $3 \times 3$, which is standard/common as it is small enough to capture local features of the image, but large enough to capture meaningful features. A stride of 1 means that the kernel moves one pixel at a time, and a padding of 1 means that the kernel is padded with 1 pixel on each side. The outcome of this is that the output image size from each layer will be the same size as the input. This is part can be confusing for new users as you need to be aware of the output size of each layer to ensure that the next layer has the correct input size. 

The second layer takes the 4 feature maps from the first layer and applies 8 kernels/filters to each of them, resulting in 8 feature maps. This extra layer is expected to learn more complex features of the image and compensates for spatial reduction from the pooling layer, which are called in between each convolutional layer after the activation function, see the forward pass of the class below.

TIP BOX

The convolutional kernel is almost always an odd number (e.g. 3, 5, 7, ...). This is because we want to have a central point for the kernel to focus on. 

In [ ]:
# define the CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding=1)
        # define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding=1)
        # define the fully connected layer
        self.fc = nn.Linear(in_features=8*7*7, out_features=10) # << See Below for explanation

    def forward(self, x):

        # First convolutional block
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # <<< Half the size of the image

        # Second convolutional block
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)  # <<< Half the size of the image

        # Flatten and pass through the fully connected layer
        x = x.flatten(start_dim=1)
        x = self.fc(x)

        return x

TIP BOX

In the previous lesson, the network was defined using `self.network = nn.Sequential(...)` in `__init__`, with `forward()` simply returning `self.network(x)`. That approach is clean and concise for a straightforward, fully sequential model.

Here we call each layer explicitly inside `forward()` instead. For *this* model the two approaches are equivalent, but the explicit style scales to more complex architectures. For example, when you need skip connections, want to return intermediate feature maps, or conditionally apply layers. The `CNN_Pathology` model in the next notebook is a good example of this. Learning the explicit pattern now means you will not need to refactor later.

The fully connected layer is the final layer of the network. It takes the output of the last convolutional layer and maps it to the number of classes in the dataset. 

In this case, the last convolutional layer has 8 channels and after 2 rounds of max pooling with a kernel and stride of 2,  the original image has been downscaled from $28 \times 28$, first to $14 \times 14$ and then to $7 \times 7$.

The output of the fully connected layer is 10, corresponding to the 10 classes in the dataset.

REMEMBER BOX

We have chosen not to place the **softmax** activation function in the network, as we will be using the PyTorch cross entropy function during the training process. This function already includes the softmax activation function, [*see previous lesson*](../AI1/testing-inference.html#mnist-classification).

## Training the CNN

For the training and testing we can utilise the functions that we defined in the [previous lesson](../AI1/testing-inference.html), as it is the same process. The data is loaded, predictions made, loss calculated, and then the weights updated during the backwards pass.

The only change that has been made is that we will now track training loss, as well as testing loss and accuracy.

In [ ]:
def train(model, device, train_loader, optimiser, epoch):
    """
    Train a neural network model for one epoch.
    
    :param model: The neural network model to train. Must have a forward() method
                  and be compatible with PyTorch's training paradigm.
    :type model: torch.nn.Module
    :param device: The device (CPU or GPU) on which to perform training computations.
                   Typically created using torch.device("cpu") or torch.device("cuda").
    :type device: torch.device
    :param train_loader: DataLoader containing the training dataset. Yields batches
                         of (data, target) tuples during iteration.
    :type train_loader: torch.utils.data.DataLoader
    :param optimiser: The optimization algorithm used to update model parameters.
                      Common choices include SGD, Adam, etc.
    :type optimiser: torch.optim.Optimizer
    :param epoch: Current epoch number (used for progress reporting).
    :type epoch: int
    
    :returns: The average training loss for the epoch.
    """
    model.train() # set the model to training mode

    # Track the training loss
    train_loss = 0.0  # <<<<< new

    for batch_idx, (data, target) in enumerate(train_loader):

        # Load the data in and its target variables
        data, target = data.to(device), target.to(device) # send the data to the device

        # Complete a forward pass and calculate the loss
        optimiser.zero_grad() # zero the gradients
        output = model(data) # forward pass
        loss = F.cross_entropy(output, target) # updated to have CrossEntropyLoss

        loss.backward() # backward pass
        optimiser.step() # update the weights

        # update training loss
        train_loss += loss.item()*data.size(0)  # <<<<< new

        # Print the training loss
        if batch_idx % 500 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    
    return train_loss/len(train_loader.dataset) 

In [ ]:
# Define a function to test the model
def test(model, device, test_loader):
    """
    Evaluate a trained neural network model on the test dataset.
    
    :param model: The trained neural network model to evaluate. Must have a forward()
                  method and be compatible with PyTorch's evaluation paradigm.
    :type model: torch.nn.Module
    :param device: The device (CPU or GPU) on which to perform evaluation computations.
                   Typically created using torch.device("cpu") or torch.device("cuda").
    :type device: torch.device
    :param test_loader: DataLoader containing the test dataset. Yields batches of
                        (data, target) tuples during iteration.
    :type test_loader: torch.utils.data.DataLoader
    
    :returns: Tuple of (average test loss, number of correct predictions, total number of test samples)
    """
    model.eval() # set the model to evaluation mode

    # Set variables to collect cumulative loss and correct predictions
    test_loss = 0
    correct = 0

    with torch.no_grad(): # don't calculate the gradients

        for data, target in test_loader:

            data, target = data.to(device), target.to(device) # send the data to the device

            # Complete a forward pass and calculate the loss
            output = model(data) # forward pass
            loss = F.cross_entropy(output, target) # updated to have CrossEntropyLoss

            # update testing loss
            test_loss += loss.item()*data.size(0)

            # Get the top prediction and check if correct
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max probability
            correct += pred.eq(target.view_as(pred)).sum().item() # Check if the prediction is correct

    # Calculate the average loss and accuracy
    test_loss /= len(test_loader.dataset)

    return test_loss, correct, len(test_loader.dataset)


Training the model for only **5 epochs** still generates very good predictions from the very start, demonstrating the power of CNNs on previously difficult datasets.

In [ ]:
# instantiate the CNN
model = CNN()

# Define the device
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device("cuda") # Defines the device you want to have data moved to
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f"Using: {device} (Cuda)")

elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using: {device} (Apple Silicon)")

else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

model = model.to(device)

In [ ]:
# define the loss function
criterion = nn.CrossEntropyLoss()

# define the optimiser
optimiser = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# define the number of epochs to train for
epochs = 5

# to track the training and test loss as the model trains
train_losses, test_losses = [], [] 

# train the model
for epoch in range(epochs):

    train_loss = train(model, device, train_loader, optimiser, epoch)
    
    test_loss, correct, total = test(model, device, test_loader)

    # save the losses for plotting
    train_losses.append(train_loss)
    test_losses.append(test_loss)

    # print training/validation statistics
    print(f'Epoch: {epoch+1} | Training Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f} | Test Accuracy: {100*correct/total:.2f}%')

## When to stop training

The plot below is very common in *machine learning*, the training and testing loss every epoch of training. Typically, you will see a rapid, exponential decline in loss during the first new epochs, eventually leading to a plateau. 

We don't want the **training loss** to be lower (or much lower) than the **testing loss**, as this is an indication of overfitting. Once the training begins to plateau it is a good idea to stop the training early, saving you time and resources, as well as preventing overfitting as the model tries to seek minimal gains.

In the next page we will demonstrate some techniques to meet these goals programmatically.

In [ ]:
x_epochs = range(1, len(train_losses) + 1)
# plot the training and validation loss
plt.plot(x_epochs, train_losses, label='Training loss')
plt.plot(x_epochs, test_losses, label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(x_epochs) 
plt.legend(frameon=False)
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# set the model to evaluation mode before making predictions
model.eval()

# obtain one batch of test images
images, labels = next(iter(test_loader))
images = images.to(device)

# get sample outputs
with torch.no_grad():
    output = model(images)

# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(10, 4))
for idx in np.arange(10):
    ax = fig.add_subplot(2, 5, idx+1, xticks=[], yticks=[])
    plt.imshow(images[idx].cpu().numpy().squeeze(), cmap='gray')
    ax.set_title(f'Pr: {preds[idx]} Tr: {labels[idx].item()}', color=("green" if preds[idx]==labels[idx].item() else "red"))

fig.suptitle("Pr == Predicted, Tr == True Labels", fontsize=16, fontweight='bold')
plt.show()

PRACTICE EXERCISE

Download the notebook and modify the `CNN` class to use `kernel_size=5` in both convolutional layers instead of `kernel_size=3`. You will also need to change `padding` from `1` to `2` to keep the output spatial dimensions the same.

Retrain for 5 epochs and compare the test accuracy and final loss to the original model. Does the larger kernel improve performance? Do you notice any difference in training time? Why might a larger kernel capture different features to a smaller one?

In [ ]:
# Write any code here:

## Challenges

Common challenges in using CNNs for image recognition include overfitting, especially when training on small datasets; the high computational cost and memory requirements for training large networks; the need for large labeled datasets; and the difficulty in interpreting the learned features and decisions of the network. Variations in object scale, orientation, and illumination in images can also pose recognition challenges.

As with all neural networks, CNNs love to have more data in the training set to learn from. In fact it has been found that CNNs trained on an extremely large, general dataset perform better than smaller, specialised datasets on the specialised task. It is thought the general models gain a better "understanding" of general rules for objects. It is therefore common to use [transfer learning](https://arxiv.org/abs/1411.1792) to leverage the knowledge gained from training on a large dataset to improve performance on a smaller, specialised dataset. Classically, these models are trained on [ImageNET](https://www.image-net.org/), a giant image dataset of over 14 million images.

Additionally, for your task at hand it may be better to use someone else's pre-trained model that has been trained on a similar task to the one you are interested in. This is especially prevalent in biology where there are several software packages utilising CNNs to perform image recognition tasks. In the final [sub-module](./pre-trained-software.html) of this lesson, we will introduce some of the most popular software packages.